In [4]:
from glob import glob
import os
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

# CNN Building Tools below-these lines are causing problems in the code for some reason
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from keras import backend as K

In [6]:
df = []
df = pd.read_csv("HAM10000_img.csv")

lesion_type_dict = {
    "nv": "Melanocytic nevi",
    "mel": "Melanoma",
    "bkl": "Benign keratosis-like lesions",
    "akiec": "Actinic keratoses",
    "vasc": "Vascular lesions",
    "df": "Dermatofibroma",
}

In [7]:
# features and target variables for train test split
features = df[["dx_type", "age", "sex", "localization", "images"]]
target = df[["dx"]]
# train test data split
x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.20, random_state=42
)
# validation and train data split
x_train, x_validate, y_train, y_validate = train_test_split(
    x_train, y_train, test_size=0.1, random_state=42
)

In [8]:
# Start of CNN Building
input_shape = (75, 100, 3)
num_classes = 7

model = Sequential()

In [ ]:
# Adding layers to the model
model.add(
    Conv2D(
        32,
        kernel_size=(3, 3),
        activation="relu",
        padding="Same",
        input_shape=input_shape,
    )
)

model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", padding="Same"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation="relu", padding="Same"))
model.add(Conv2D(64, (3, 3), activation="relu", padding="Same"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(Dense(num_classes, activation="softmax"))
model.summary()